Planning architecture involves an agent that explicitly breaks a complex goal into a detailed sequence of sub-tasks before beginning execution. The output phase is a concrete, step-by-step plan that agent then follows methodically to reach the solution


In [1]:

!pip3 install -q -U langchain-openai langchain langgraph rich python-dotenv tavily-python  langchain_community langchain-tavily


In [ ]:
import os 
from dotenv import load_dotenv
from typing import TypedDict,List,Annotated,Optional

#Langchain Components
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage,ToolMessage
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import SystemMessage
from pydantic import BaseModel,Field
from langchain_tavily import TavilySearch
#Langgraph cmponents
from langgraph.graph import StateGraph,END
from langgraph.graph.message import AnyMessage,add_messages
from langgraph.prebuilt import ToolNode,tools_condition

#For pretty printing
from rich.console import Console
from rich.markdown import Markdown
console = Console()


os.environ['TAVILY_API_KEY']=""
os.environ['NEBIUS_API_KEY'] = ''
os.environ['LANGCHAIN_API_KEY'] = ''

# CRITICAL: Map NEBIUS_API_KEY to OPENAI_API_KEY for ChatNebius
# ChatNebius requires OPENAI_API_KEY, not NEBIUS_API_KEY
os.environ['OPENAI_API_KEY'] = ''

Rebuilding the ReAct Agent

In [16]:
console= Console()

# Define the state for our graphs
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# 1. Define the base tool from the tavily package
tavily_search_tool = TavilySearch(max_results=2)

# 2. THE FIX: Simplify the custom tool. 
#    The .invoke() method already returns a clean string, so we just pass it through.
@tool
def web_search(query: str) -> str:
    """Performs a web search using Tavily and returns the results as a string."""
    console.print(f"--- TOOL: Searching for '{query}'...")
    try:
        results = tavily_search_tool.invoke(query)
        return results
    except Exception as e:
        error_msg = f"Search failed: {str(e)}"
        console.print(f"[red]{error_msg}[/red]")
        return error_msg

# 3. Define the LLM and bind it to our custom tool
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools([web_search])

# 4. Agent node with a system prompt to force one tool call at a time
def react_agent_node(state: AgentState):
    console.print("--- REACTIVE AGENT: Thinking... ---")
    
    messages_with_system_prompt = [
        SystemMessage(content="You are a helpful research assistant. You must call one and only one tool at a time. Do not call multiple tools in a single turn. After receiving the result from a tool, you will decide on the next step.")
    ] + state["messages"]

    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    try:
        response = llm_with_tools.invoke(messages_with_system_prompt)
        return {"messages": [response]}
    except Exception as e:
        console.print(f"[red]Agent error: {e}[/red]")
        raise

# 5. Use our corrected custom tool in the ToolNode
tool_node = ToolNode([web_search])

# The ReAct graph with its characteristic loop
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", tool_node)
react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", tools_condition)
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("Reactive (ReAct) agent compiled successfully.")

Reactive (ReAct) agent compiled successfully.


In [4]:
plan_centric_query = """
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
"""

console.print(f"[bold yellow]Testing REACTIVE agent on a plan-centric query:[/bold yellow] '{plan_centric_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", plan_centric_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]Current State Update[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold red]Final Output from Reactive Agent[/bold red] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

Testing REACTIVE agent on a plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

--- Current State Update ---

================================ Human Message =================================


Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.



--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

...

--- REACTIVE AGENT: Thinking... ---

--- Current State Update ---

================================== Ai Message ==================================

The current populations of the capital cities are as follows:

- **Paris, France**: Approximately 11,347,000 (metro area)
- **Berlin, Germany**: Approximately 3,580,000 (metro area)
- **Rome, Italy**: Approximately 4,347,000 (metro area)

Now, let's calculate the combined total population of these three cities:

\[
\text{Combined Total} = 11,347,000 + 3,580,000 + 4,347,000
\]

Calculating this gives:

\[
\text{Combined Total} = 19,274,000
\]

Next, we need to compare this combined total to the population of the United States, which is approximately 331 million (as of 2023). 

Now, let's compare:

- **Combined Total of Paris, Berlin, and Rome**: 19,274,000
- **Population of the United States**: 331,000,000

Clearly, the population of the United States is larger than the combined total of the capital cities of France, Germany, and Italy.


--- Final Output from Reactive Agent ---

The current populations of the capital cities are as follows:                                                      

 • Paris, France: Approximately 11,347,000 (metro area)                                                            
 • Berlin, Germany: Approximately 3,580,000 (metro area)                                                           
 • Rome, Italy: Approximately 4,347,000 (metro area)                                                               

Now, let's calculate the combined total population of these three cities:                                          

[ \text{Combined Total} = 11,347,000 + 3,580,000 + 4,347,000 ]                                                     

Calculating this gives:                                                                                            

[ \text{Combined Total} = 19,274,000 ]                                                                             

Next, we need to compare this combined total to the population of the United States, which is approximately 331    
million (as of 2023).                                                                                              

Now, let's compare:                                                                                                

 • Combined Total of Paris, Berlin, and Rome: 19,274,000                                                           
 • Population of the United States: 331,000,000                                                                    

Clearly, the population of the United States is larger than the combined total of the capital cities of France,    
Germany, and Italy.

#Advacned Planner

In [5]:
import re
class Plan(BaseModel):
    """ A Plan of tool calls to execute to answer the user's query"""
    steps: List[str]=Field(description="A list of tool cals taht, wehn executed, will answer the query")

# Deine the state for planning agent
class PlanningStage(TypedDict):
    user_request:str
    plan:Optional[List[str]]
    intermediate_steps:List[ToolMessage]
    final_answer:Optional[str]

def planner_node(state:PlanningStage):
    """Generates a plan of action to answer the users request"""
    console.print("-- PLANNER: Decomposing task")
    planner_llm=llm.with_structured_output(Plan)

    prompt = f"""You are an expert planner. Your job is to create a step-by-step plan to answer the user's request.
        Each step in the plan must be a single call to the `web_search` tool.

        **Instructions:**
        1. Analyze the user's request.
        2. Break it down into a sequence of simple, logical search queries.
        3. Format the output as a list of strings, where each string is a single valid tool call.

        **Example:**
        Request: "What is the capital of France and what is its population?"
        Correct Plan Output:
        [
            "web_search('capital of France')",
            "web_search('population of Paris')"
        ]

        **User's Request:**
         {state['user_request']}
    """

    plan_result=planner_llm.invoke(prompt)
    console.print(f"---- PLANNER: Generated Plan:{plan_result.steps}--")
    #Use plan_request, not plan.steps to avaoid confusion with the variable naem plan
    return {"plan":plan_result.steps}

def executor_node(state:PlanningStage):
    """Executes the next step in the plan"""
    plan=state["plan"]
    next_step=plan[0]

    match = re.search(r"(\w+)\((?:\"|\')(.*?)(?:\"|\')\)", next_step)

    if not match:
        tool_name = "web_search"
        query = next_step
    else:
        tool_name, query = match.groups()[0], match.groups()[1]
    
    console.print(f"--- EXECUTOR: Calling tool '{tool_name}' with query '{query}' ---")
    
    result = tavily_search_tool.invoke(query)
    
    # We still create a ToolMessage, but the tool call itself is now safe.
    tool_message = ToolMessage(
    content=str(result),
    name=tool_name,
    tool_call_id=f"manual-{hash(query)}"
    )
    
    return {
        "plan": plan[1:], # Pop the executed step from the plan
        "intermediate_steps": state["intermediate_steps"] + [tool_message]
    }

def synthesizer_node(state: PlanningStage):
    """Synthesizes the final answer from the intermediate steps."""
    console.print("--- SYNTHESIZER: Generating final answer... ---")
    
    context = "\n".join([f"Tool {msg.name} returned: {msg.content}" for msg in state["intermediate_steps"]])
    
    prompt = f"""You are an expert synthesizer. Based on the user's request and the collected data, provide a comprehensive final answer.
    
    Request: {state['user_request']}
    Collected Data:
    {context}
    """
    final_answer = llm.invoke(prompt).content
    return {"final_answer": final_answer}

print("Planner, Executor, and Synthesizer nodes defined.")

Planner, Executor, and Synthesizer nodes defined.


In [10]:
def planning_router(state:PlanningStage):
    if not state['plan']:
        console.print("--- ROUTER: Plan complete. Moving to syntesizer ---")
        return "synthesize"
    else:
        console.print("--ROUTER: Plan has more steps. Continuing execution")
        return "execute"
    

planning_graph_builder=StateGraph(PlanningStage)
planning_graph_builder.add_node("plan",planner_node)
planning_graph_builder.add_node("execute", executor_node)
planning_graph_builder.add_node("synthesize", synthesizer_node)

planning_graph_builder.set_entry_point("plan")
planning_graph_builder.add_conditional_edges("plan", planning_router, {"execute": "execute", "synthesize": "synthesize"}) # Route after planning
planning_graph_builder.add_conditional_edges("execute", planning_router, {"execute": "execute", "synthesize": "synthesize"})
planning_graph_builder.add_edge("synthesize", END)

planning_agent_app = planning_graph_builder.compile()
print("Planning agent compiled successfully.")


Planning agent compiled successfully.


In [11]:
console.print(f"[bold green]Testing PLANNING agent on the same plan-centric query:[/bold green] '{plan_centric_query}'\n")

# Remember to initialize the state correctly, especially the list for intermediate steps
initial_planning_input = {"user_request": plan_centric_query, "intermediate_steps": []}

final_planning_output = planning_agent_app.invoke(initial_planning_input)

console.print("\n--- [bold green]Final Output from Planning Agent[/bold green] ---")
console.print(Markdown(final_planning_output['final_answer']))

Testing PLANNING agent on the same plan-centric query: '
Find the population of the capital cities of France, Germany, and Italy. 
Then calculate their combined total. 
Finally, compare that combined total to the population of the United States, and say which is larger.
'

-- PLANNER: Decomposing task

---- PLANNER: Generated Plan:["web_search('population of Paris')", "web_search('population of Berlin')", 
"web_search('population of Rome')", "web_search('population of United States')"]--

--ROUTER: Plan has more steps. Continuing execution

--- EXECUTOR: Calling tool 'web_search' with query 'population of Paris' ---

...

--- EXECUTOR: Calling tool 'web_search' with query 'population of United States' ---

--- ROUTER: Plan complete. Moving to syntesizer ---

--- SYNTHESIZER: Generating final answer... ---

--- Final Output from Planning Agent ---

Based on the collected data, here are the populations of the capital cities of France, Germany, and Italy:         

 1 Paris, France: The population is approximately 11,346,800 (2025 estimate).                                      
 2 Berlin, Germany: The population is approximately 3,580,190 (2025 estimate).                                     
 3 Rome, Italy: The population is approximately 4,332,000 (2024 estimate).                                         

                                  Combined Total Population of the Capital Cities                                  

To find the combined total population of these three cities, we add their populations together:                    

 • Paris: 11,346,800                                                                                               
 • Berlin: 3,580,190                                                                                               
 • Rome: 4,332,000                                                                                                 

Combined Total = 11,346,800 + 3,580,190 + 4,332,000 = 19,258,990                                                   

                                Comparison with the Population of the United States                                

The population of the United States is estimated to be approximately 341.2 million (2024 estimate).                

                                                    Conclusion                                                     

Comparing the combined total population of the capital cities (19,258,990) to the population of the United States  
(341,200,000):                                                                                                     

 • 341,200,000 (U.S.) is significantly larger than 19,258,990 (combined total of Paris, Berlin, and Rome).         

Thus, the population of the United States is larger.

In [12]:
class ProcessEvaluation(BaseModel):
    """Schema for evaluating an agent's problem-solving process."""
    task_completion_score: int = Field(description="Score 1-10 on whether the agent successfully completed the task.")
    process_efficiency_score: int = Field(description="Score 1-10 on the efficiency and directness of the agent's process. A higher score means a more logical and less roundabout path.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ProcessEvaluation)

def evaluate_agent_process(query: str, final_state: dict):
    # For the ReAct agent, the trace is in 'messages'. For Planning, it's in 'intermediate_steps'.
    if 'messages' in final_state:
        trace = "\n".join([f"{m.type}: {str(m.content)}" for m in final_state['messages']])
    else:
        trace = f"Plan: {final_state.get('plan', [])}\nSteps: {final_state.get('intermediate_steps', [])}"
        
    prompt = f"""You are an expert judge of AI agents. Evaluate the agent's process for solving the task on a scale of 1-10.
    Focus on whether the process was logical and efficient.
    
    **User's Task:** {query}
    **Full Agent Trace:**\n```\n{trace}\n```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Reactive Agent's Process ---")
react_agent_evaluation = evaluate_agent_process(plan_centric_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

console.print("\n--- Evaluating Planning Agent's Process ---")
planning_agent_evaluation = evaluate_agent_process(plan_centric_query, final_planning_output)
console.print(planning_agent_evaluation.model_dump())

--- Evaluating Reactive Agent's Process ---

{
    'task_completion_score': 10,
    'process_efficiency_score': 9,
    'justification': 'The agent successfully completed the task by accurately finding the populations of the 
capital cities of France, Germany, and Italy, calculating their combined total, and comparing it to the population 
of the United States. The process was logical and followed a clear sequence, although there were some redundant 
queries for the same information which slightly affected efficiency.'
}

--- Evaluating Planning Agent's Process ---

{
    'task_completion_score': 8,
    'process_efficiency_score': 7,
    'justification': 'The agent successfully retrieved the populations of Paris, Berlin, and Rome, and also found 
the population of the United States. However, the agent did not explicitly show the calculation of the combined 
total or the comparison step in the trace, which slightly detracts from the clarity of the process. The efficiency 
is good, but there were some unnecessary steps in retrieving multiple sources for the same information, which could
have been streamlined.'
}